In [2]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [3]:
filename = "ABox.ttl"

In [4]:
text1 = '''CQ_2.1
Return the risk summaries and the agents of deterioration of the risks assigned to the "Battistero degli Ariani".
'''

query1 = '''
PREFIX tbox: <http://purl.org/sirius/ontology/development/02/schema/>
PREFIX abox: <http://purl.org/sirius/ontology/development/02/data/>

SELECT ?risk ?risk_summary ?agent_of_deterioration
WHERE {
    ?risk_assessment a tbox:RiskAssessment ;
                    tbox:assignsRiskTo abox:battistero-degli-ariani ;
                    tbox:assigns ?risk .
    ?risk a tbox:Risk ;
        tbox:hasSummary ?risk_summary ;
        tbox:isEmbodiedBy ?agent_of_deterioration .
}
'''

In [8]:
text2 = '''CQ_2.2
Return the hazards originating the risk that are related to the agent of deterioration "Physical forces".
'''

query2 = '''
PREFIX tbox: <http://purl.org/sirius/ontology/development/02/schema/>
PREFIX abox: <http://purl.org/sirius/ontology/development/02/data/>

SELECT ?risk ?hazard
WHERE {
    ?risk a tbox:Risk ;
            tbox:originatesFrom ?hazard ;
            tbox:isEmbodiedBy tbox:physical-forces .
}
'''

In [11]:
text3 = '''CQ_2.3
Return the adverse effects of the risk that is related to the agents of deterioration "Pests", "Water" and "Fire".
'''

query3 = '''
PREFIX tbox: <http://purl.org/sirius/ontology/development/02/schema/>
PREFIX abox: <http://purl.org/sirius/ontology/development/02/data/>

SELECT ?risk ?agent_of_deterioration ?adverse_effect
WHERE {
    ?risk a tbox:Risk ;
            tbox:causes ?adverse_effect ;
            tbox:isEmbodiedBy ?agent_of_deterioration .
    FILTER (
    ?agent_of_deterioration = tbox:pests || 
    ?agent_of_deterioration = tbox:water || 
    ?agent_of_deterioration = tbox:fire
    )
}
'''

In [12]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_2.1
Return the risk summaries and the agents of deterioration of the risks assigned to the "Battistero degli Ariani".

+-------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------+
| risk                                                                                | risk_summary                                                                                                                                                  | agent_of_deterioration                                                    |
|-------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------